In [17]:
using Polyhedra, CDDLib
using DrakeVisualizer
import ColorTypes: RGBA
using PyCall
path_to_my_module = "/home/manuelli/drake-mpc/src/boxrobots/src"
push!(LOAD_PATH, path_to_my_module)
reload("BoxRobots")
import BoxRobots
using DrakeVisualizer
import ColorTypes: RGBA
using Polyhedra: SimpleHRepresentation

br = BoxRobots

BoxRobots

In [18]:
# unshift!(PyVector(pyimport("sys")["path"]), "../")
@pyimport boxatlas
@pyimport boxatlas.boxatlascontroller as boxatlascontroller
@pyimport boxatlas.contactstabilizationutils as csu
CSU = csu.ContactStabilizationUtils

PyObject <class boxatlas.contactstabilizationutils.ContactStabilizationUtils at 0x7f87355f7668>

In [19]:
# attempt to visualize a state
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window();
vis = Visualizer()
delete!(vis)
env, robot = br.make_robot_and_environment()
robot_state = br.make_robot_state()
robot_state.centroidal_dynamics_state.vel[1] = -1.75
br.draw_environment(vis, env)
mass = robot.mass
gravity = robot.gravity
vis_options = br.BoxRobotVisualizerOptions(force_arrow_normalizer=mass*abs(gravity[end]))

BoxRobots.BoxRobotVisualizerOptions
  force_arrow_normalizer: Float64 98.0
  com_radius: Float64 0.1
  contact_point_radius: Float64 0.05
  playback_dt: Float64 0.05


# QPInnerLoopController

In [20]:
# setup MIQP controller so we can get python controller out
miqp_controller = br.MIQPController()
robot_state = br.make_robot_state()
robot_state.centroidal_dynamics_state.vel[1] = -1.75
python_controller = miqp_controller.python_controller
robot_python = python_controller[:robot]
state_python = br.convert_box_robot_state_to_python(robot_python, robot_state)
opt = python_controller[:construct_contact_stabilization_optimization](state_python)
soln_data = pycall(opt[:solve], PyObject)

Optimize a model with 2372 rows, 680 columns and 5958 nonzeros
Model has 1188 quadratic objective terms
Coefficient statistics:
  Matrix range    [2e-03, 1e+03]
  Objective range [2e+00, 2e+03]
  Bounds range    [5e-01, 1e+03]
  RHS range       [2e-01, 1e+03]
Presolve removed 718 rows and 56 columns
Presolve time: 0.01s
Presolved: 1654 rows, 624 columns, 4096 nonzeros
Presolved model has 1140 quadratic objective terms
Variable types: 546 continuous, 78 integer (78 binary)

Root relaxation: objective -1.398097e+03, 2196 iterations, 0.11 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -1398.0969    0   74          - -1398.0969      -     -    0s
H    0     0                    -1309.332960 -1398.0969  6.78%     -    0s
     0     2 -1398.0969    0   74 -1309.3330 -1398.0969  6.78%     -    0s
*  379   122              72    -1385.710432 -1391.3730  0.41%   7.1    0

PyObject SolutionData(opt=<boxatlas.contactstabilization.BoxAtlasContactStabilization object at 0x7f8778374e90>, states=<utils.trajectory.Trajectory object at 0x7f872c3a9a50>, inputs=<utils.trajectory.Trajectory object at 0x7f872c2b98d0>, contact_indicator=[<utils.piecewise.Piecewise object at 0x7f872c2ed590>, <utils.piecewise.Piecewise object at 0x7f872c2a6d50>, <utils.piecewise.Piecewise object at 0x7f872c2b9150>, <utils.piecewise.Piecewise object at 0x7f872c2b9c90>], ts=array([ 0.  ,  0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,  0.35,  0.4 ,
        0.45,  0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,
        0.9 ,  0.95,  1.  ]), solve_time=0.8149540424346924)

# Simulate QPInnerLoopController

In [21]:
t_start = 0.0
dt = 0.05
duration = 1.5
qp_controller = br.QPInnerLoopController(python_controller, t_start,
    robot_state, soln_data)
@time traj = br.simulate_tspan(robot, qp_controller, robot_state, dt, duration)
br.playback_trajectory(vis, traj; options=vis_options)

contact assignments passed in
Optimize a model with 2452 rows, 680 columns and 6038 nonzeros
Model has 1188 quadratic objective terms
Coefficient statistics:
  Matrix range    [2e-03, 1e+03]
  Objective range [2e+00, 2e+03]
  Bounds range    [5e-01, 1e+03]
  RHS range       [2e-01, 1e+03]
Presolve removed 1943 rows and 337 columns
Presolve time: 0.00s
Presolved: 509 rows, 343 columns, 1367 nonzeros
Presolved model has 746 quadratic objective terms
Variable types: 343 continuous, 0 integer (0 binary)

Root relaxation: objective -1.385985e+03, 832 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    -1385.984586 -1385.9846  0.00%     -    0s

Explored 0 nodes (832 simplex iterations) in 0.04 seconds
Thread count was 6 (of 12 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective -1.385984586374e+03, best bou

  7.607124 seconds (644.82 k allocations: 26.010 MB, 0.11% gc time)


In [23]:
br.playback_trajectory(vis, traj, options=vis_options, playback_speed=1.0)

In [25]:
br.slider_playback(vis, traj, options=vis_options)

Interact.Options{:SelectionSlider,Int64}(input: Signal{Int64}(16, nactions=1),"idx",16,"16",Interact.OptionDict(DataStructures.OrderedDict("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…),Dict(18=>"18",30=>"30",2=>"2",16=>"16",11=>"11",21=>"21",7=>"7",9=>"9",25=>"25",10=>"10"…)),Any[],Any[],true,"horizontal")

nothing

# Simulate MIQP Controller

In [ ]:

miqp_controller = br.MIQPController()
robot_state = br.make_robot_state()
robot_state.centroidal_dynamics_state.vel[1] = -1.75
python_controller = miqp_controller.python_controller
robot_python = python_controller[:robot]
state_python = br.convert_box_robot_state_to_python(robot_python, robot_state)
opt = python_controller[:construct_contact_stabilization_optimization](state_python)
soln_data = pycall(opt[:solve], PyObject)

# construct the SimpleInnerLoopController
t_start = 0.
inner_loop_controller = br.SimpleInnerLoopController(t_start, soln_data)

t = 0.0
dt = 0.025
duration = 1.0
@time traj = br.simulate_tspan(robot, inner_loop_controller, robot_state, dt, duration)
br.playback_trajectory(vis, traj; options=vis_options)


In [ ]:
# extract solve times from the trajectory of SimulationData
function get_solve_time(data)
    return data.controller_data.solve_time
end

function get_contact_array(data)
    return data.state.limb_states[:left_hand].in_contact 
end

solve_time_array = get_solve_time.(traj.data)
contact_array = get_contact_array.(traj.data)

In [ ]:
using Plots
# Use the GR backend for Plots.jl, because it's fast
gr()

In [ ]:
plot(traj.time, solve_time_array, xlabel="time", label="solve time")
# plot!(traj.time, contact_array, label="contact indicator")